In [1]:
import sys
sys.path.append("../Jordan dashboard")
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import os
import pickle
import plotly.io as pio
import scripts.read_data as rd

pio.templates.default = "plotly_white"

In [31]:
path = '../Jordan dashboard'
scenarios = [
    'Reference', 
    'New Resources',
    'Increased Water Productivity',
#     'Improve AG Eff by 20percent',
    'Reduce NRW to 20 percent'
]
climates = [
    'Historical Trend', 
    'Climate Change'
]

files = ['results.gz', 'wwtp_data.gz', 'desal_data.gz',
         'summary_results.gz', 'production.gz']

all_results = [pd.DataFrame()] * 7
for scenario in scenarios:
    for climate in climates:
        results = rd.load_data(path, scenario, [climate], '', 'all')
#         _df_delivered, _df_required, _df_gw, _df_pipelines, _df_wwtp, _df_desal, _df_crop = results
        
        for dff in results: 
            dff['Scenario'] = scenario
            dff['Climate'] = climate
        
        for i, dff in enumerate(results):
            all_results[i] = all_results[i].append(dff, ignore_index=True)

df_delivered = all_results[0]
df_required = all_results[1]
df_gw = all_results[2]
df_pipelines = all_results[3]
df_wwtp = all_results[4]
df_desal = all_results[5]
df_crop = all_results[6]

In [32]:
names = {'sswd': 'Water delivered (Mm<sup>3</sup>)', 'production_kg': 'Crop production (ton)', 
         'swpa_e': 'Energy demand (GWh)', 'unmet_demand_year': 'Annual unmet demand (%)',
         'wtd': 'Depth to groundwater (meters)'}

layout = dict(
    autosize=True,
    height=500,
    margin=dict(l=0, r=0, b=0, t=50),
    hovermode="closest",
    plot_bgcolor="#fff",
    paper_bgcolor="#fff",
    legend=dict(font=dict(size=16), orientation="h", title=''),
    xaxis={'title': ''},
    showlegend=True,
    hoverlabel={'align': 'left'},
    font=dict(size=18, color="#7f7f7f"),
#     yaxis={'title': ''},
)

category_orders = {'Climate': ['Historical Trend', 'Climate Change'],
                   'Scenario': ['Reference', 'Desalination'],
                   'type': ['Groundwater pumping', 'Surface water conveyance', 'Wastewater treatment',
                            'Desalination', 'Desalinated water conveyance']}

def update_labels(fig):
    for axis in fig.layout:
        if type(fig.layout[axis]) == go.layout.XAxis:
            fig.layout[axis].title.text = ''
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

In [33]:
dff = df_delivered
dff = dff.groupby(['Scenario', 'Climate', 'Year']).agg({'value': lambda r: sum(r) / 1000000}).reset_index()

fig = px.line(dff, x='Year', y='value', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names,  category_orders=category_orders)
fig.update_layout(layout, yaxis_title_text='Water delivered (Mm<sup>3</sup>)')
update_labels(fig)
fig.show()
fig.write_image("High level results/waterDeliveries.png", scale=4, width=900, height=500)
fig.write_html("High level results/waterDeliveries.html", include_plotlyjs='cdn')

In [38]:
dff_delivered = df_delivered.groupby(['Scenario', 'Climate', 'Year', 'type'])['value'].sum() / 1000000
dff_delivered = dff_delivered.reset_index()
dff_required = df_required.groupby(['Scenario', 'Climate', 'Year', 'type'])['value'].sum() / 1000000
dff_required = dff_required.reset_index()

dff_unmet = dff_required.copy()
dff_unmet['value'] = (dff_unmet.value - dff_unmet.set_index(['Scenario', 'Climate', 'Year', 'type']).index.map(
    dff_delivered.set_index(['Scenario', 'Climate', 'Year', 'type']).value)) / dff_unmet.value

sector = 'Agriculture'
dff = dff_unmet.reset_index()
dff = dff.loc[dff['type'] == sector]

fig = px.line(dff, x='Year', y='value', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout, yaxis=dict(tickformat='%', title_text='Annual unmet demand (%)'))
update_labels(fig)
fig.show()
fig.write_image(f"High level results/unmetDemand{sector}.png", scale=4, width=900, height=500)
fig.write_html(f"High level results/unmetDemand{sector}.html", include_plotlyjs='cdn')

In [35]:
dff = df_crop.groupby(['Scenario', 'Climate', 'Year']).agg({'value': lambda r: sum(r) / 1000000}).reset_index()

fig = px.line(dff, x='Year', y='value', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10, 
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout, yaxis_title_text='Crop production (kton)')
update_labels(fig)
fig.show()
fig.write_image("High level results/cropProduction.png", scale=4, width=900, height=500)
fig.write_html("High level results/cropProduction.html", include_plotlyjs='cdn')

In [36]:
dff_delivered = df_delivered.loc[df_delivered['type']=='Agriculture']
dff_delivered = dff_delivered.groupby(['Scenario', 'Climate', 'Year']).agg({'value': lambda r: sum(r) / 1000000}).reset_index()
dff_crop = df_crop.groupby(['Scenario', 'Climate', 'Year']).agg({'value': lambda r: sum(r) / 1000000}).reset_index()
dff = dff_delivered[['Scenario', 'Climate', 'Year']]
dff['value'] = dff_crop['value'] / dff_delivered['value']

fig = px.line(dff, x='Year', y='value', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10, 
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout, yaxis_title_text='Crop per drop (kg/m<sup>3</sup>)')
update_labels(fig)
fig.show()
fig.write_image("High level results/cropPerDrop.png", scale=4, width=900, height=500)
fig.write_html("High level results/cropPerDrop.html", include_plotlyjs='cdn')

In [37]:
dff_energy = pd.DataFrame()

for df in [df_wwtp, df_desal, df_gw, df_pipelines]:
    dff = df.groupby(['Scenario', 'Climate', 'Year', 'type'])['SWPA_E_'].sum() / (1000000)
    dff = dff.reset_index()
    dff_energy = dff_energy.append(dff, sort=False)
dff_energy.rename(columns={'SWPA_E_': 'value'}, inplace=True)

dff = dff_energy.groupby(['Scenario', 'Climate', 'Year']).agg({'value': 'sum'}).reset_index()

fig = px.line(dff, x='Year', y='value', color='Scenario', facet_col='Climate',
              color_discrete_sequence=px.colors.qualitative.T10,
              facet_col_spacing=0.06, labels=names, category_orders=category_orders)
fig.update_layout(layout, yaxis_title_text='Energy demand (GWh)')
update_labels(fig)
fig.show()
fig.write_image("High level results/energyDemand.png", scale=4, width=900, height=500)
fig.write_html("High level results/energyDemand.html", include_plotlyjs='cdn')